In [63]:
# reference: TensorFlow tutorials

import os
import glob
import numpy as np
import cv2
from sklearn.utils import shuffle
import math
import random
import pandas as pd
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

from sklearn.metrics import confusion_matrix
from datetime import timedelta

In [64]:
# load image
classes =  ['dogs', 'cats']
train_path = '/Users/miezai/Downloads/tensorflow-image-classification-master/train_400/'
test_path = '/Users/miezai/Downloads/tensorflow-image-classification-master/test/'
image_size = 128

def load_image(train_path, image_size, classes):
    images = []
    labels = []
    ids = []
    cls = []

    for fld in classes:
        index = classes.index(fld)
        path = os.path.join(train_path, fld, '*g')
        files = glob.glob(path)
    
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            flbase = os.path.basename(fl)
            ids.append(flbase)
            cls.append(fld)

    images = np.array(images)
    labels = np.array(labels)
    ids = np.array(ids)
    cls = np.array(cls)
    print("Reading train images")
    
    return images, labels, ids, cls


In [65]:
# helpful function from TensorFlow tutorials
class DataSet(object):
    def __init__(self, images, labels, ids, cls):
        """Construct a DataSet. one_hot arg is used only if fake_data is true."""

        self._num_examples = images.shape[0]
        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 255.0)

        self._images = images
        self._labels = labels
        self._ids = ids
        self._cls = cls
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def ids(self):
        return self._ids

    @property
    def cls(self):
        return self._cls

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size

        if self._index_in_epoch > self._num_examples:
            self._epochs_completed += 1

            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch

        return self._images[start:end], self._labels[start:end], self._ids[start:end], self._cls[start:end]

In [66]:
def read_image(train_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass  
    data_sets = DataSets()

    images, labels, ids, cls = load_image(train_path, image_size, classes)
    images, labels, ids, cls = shuffle(images, labels, ids, cls)  # shuffle the data
        
    if isinstance(validation_size, float):
        validation_size = int(validation_size * images.shape[0])

    validation_images = images[:validation_size]
    validation_labels = labels[:validation_size]
    validation_ids = ids[:validation_size]
    validation_cls = cls[:validation_size]

    train_images = images[validation_size:]
    train_labels = labels[validation_size:]
    train_ids = ids[validation_size:]
    train_cls = cls[validation_size:]

    data_sets.train = DataSet(train_images, train_labels, train_ids, train_cls)
    data_sets.valid = DataSet(validation_images, validation_labels, validation_ids, validation_cls)
    
    return data_sets

In [67]:
# test
def load_test(test_path, image_size):
    path = os.path.join(test_path, '*g')
    files = sorted(glob.glob(path))
    
    X_test = []
    X_test_id = []
    print("Reading test images")
    
    for fl in files:
        flbase = os.path.basename(fl)
        img = cv2.imread(fl)
        img = cv2.resize(img, (image_size, image_size), cv2.INTER_LINEAR)
        X_test.append(img)
        X_test_id.append(flbase)

    X_test = np.array(X_test, dtype=np.uint8)
    X_test = X_test.astype('float32')
    X_test = X_test / 255

    return X_test, X_test_id

def read_test_set(test_path, image_size):  
    images, ids  = load_test(test_path, image_size)
    return images, ids

In [68]:
validation_size = 0.05
data = read_image(train_path, image_size, classes, validation_size=validation_size)
test_images, test_ids = read_test_set(test_path, image_size)

Reading train images
Reading test images


In [69]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Validation-set:\t{}".format(len(data.valid.labels)))
print("- Test-set:\t\t{}".format(len(test_images)))

Size of:
- Training-set:		1874
- Validation-set:	98
- Test-set:		150


In [70]:
#data.test.cls = np.argmax(data.test.labels, axis=1)

In [71]:
# create weights & biases
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [72]:
#helpful functions - define layer - from TensorFlow tutorial
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  

    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    biases = create_biases(num_filters)
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
 
    layer += biases
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    layer = tf.nn.relu(layer)
    return layer

def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
 
    return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer


In [73]:
# Convolutional Layer 1.
filter_size1 = 3 
num_filters1 = 32

# Convolutional Layer 2.
filter_size2 = 3
num_filters2 = 32
# Convolutional Layer 3.
filter_size3 = 3
num_filters3 = 64

# Parameters
fc_size = 128            
num_channels = 3
img_size_flat = image_size * image_size * num_channels
img_shape = (image_size, image_size)
num_classes = len(classes)
batch_size = 32

x = tf.placeholder(tf.float32, shape=[None, image_size,image_size,num_channels], name='x')
x_image = tf.reshape(x, [-1, image_size, image_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [74]:
# create layers
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size1,
               num_filters=num_filters1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters1,
               conv_filter_size=filter_size2,
               num_filters=num_filters2)
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters2,
               conv_filter_size=filter_size3,
               num_filters=num_filters3)

layer_flat = create_flatten_layer(layer_conv3)
layer_shape = layer_conv3.get_shape()
num_features = layer_shape[1:4].num_elements()

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=num_features,
                     num_outputs=fc_size,
                     use_relu=True)
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_size,
                     num_outputs=num_classes,
                     use_relu=False)

In [75]:
# Predicted Class & Optimization 
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, dimension=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [76]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [77]:
session = tf.Session()
session.run(tf.initialize_all_variables())

train_batch_size = batch_size

In [ ]:
total_iterations = 0

def print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%}, Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

def optimize(num_iterations):
    global total_iterations
    start_time = time.time()
    
    best_val_loss = float("inf")
    patience = 0

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)

        #x_batch = x_batch.reshape(train_batch_size, img_size_flat)
        #x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        feed_dict_validate = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_validate)
            epoch = int(i / int(data.train.num_examples/batch_size))
            
            print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss)
            
            #if early_stopping:    
             #   if val_loss < best_val_loss:
              #      best_val_loss = val_loss
               #     patience = 0
                #else:
                    #patience += 1

                #if patience == early_stopping:
                 #   break

    total_iterations += num_iterations
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time elapsed: " + str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
optimize(num_iterations=9999)

Epoch 1 --- Training Accuracy:  59.4%, Validation Accuracy:  43.8%, Validation Loss: 0.779
Epoch 2 --- Training Accuracy:  40.6%, Validation Accuracy:  43.8%, Validation Loss: 0.700
Epoch 3 --- Training Accuracy:  65.6%, Validation Accuracy:  65.6%, Validation Loss: 0.687
Epoch 4 --- Training Accuracy:  68.8%, Validation Accuracy:  65.6%, Validation Loss: 0.670
Epoch 5 --- Training Accuracy:  68.8%, Validation Accuracy:  59.4%, Validation Loss: 0.668
Epoch 6 --- Training Accuracy:  62.5%, Validation Accuracy:  62.5%, Validation Loss: 0.664
Epoch 7 --- Training Accuracy:  59.4%, Validation Accuracy:  68.8%, Validation Loss: 0.630
Epoch 8 --- Training Accuracy:  50.0%, Validation Accuracy:  59.4%, Validation Loss: 0.625
Epoch 9 --- Training Accuracy:  56.2%, Validation Accuracy:  56.2%, Validation Loss: 0.628
Epoch 10 --- Training Accuracy:  59.4%, Validation Accuracy:  71.9%, Validation Loss: 0.589
Epoch 11 --- Training Accuracy:  65.6%, Validation Accuracy:  75.0%, Validation Loss: 0.5

In [ ]:
def print_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):
    num_test = len(data.valid.images)
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        j = min(i + batch_size, num_test)

        images = data.valid.images[i:j, :]
        #.reshape(batch_size, img_size_flat)
        labels = data.valid.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j

    cls_true = np.array(data.valid.cls)
    cls_pred = np.array([classes[x] for x in cls_pred]) 

    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()

    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%}"
    print(msg.format(acc, correct_sum, num_test))


In [ ]:
print_accuracy(show_example_errors=False)